In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
from langdetect import detect
import  fuzzywuzzy.fuzz as fuzz
import time
from langdetect import detect

In [1]:
import pandas as pd

In [35]:
df=pd.read_csv("IndianFoodDatasetCSV.csv")
pd.set_option('display.max_rows', 500)

In [36]:
df.drop(["RecipeName","Ingredients","Instructions"],axis=1,inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [37]:
start=time.time()
df["lang"]=df["TranslatedIngredients"].apply(lambda x:detect(x))
end = time.time()
print(end - start)      

68.83786153793335


In [38]:
df=df[df["lang"]=="en"]

In [39]:
df["lang"].value_counts()

en    6276
Name: lang, dtype: int64

In [40]:
df.to_csv('df-lan.csv',index=False)

In [41]:
df=pd.read_csv("df-lan.csv")

In [42]:
def preprocess(text):
    text=re.sub(r'[0-9]+', '', text)
    delword="Teaspoon,teaspoon,teaspoons,-,/,to taste,as needed,shredded,cup,cups,washed,sliced,finely,few,chopped,tablespoons,\
    Fresh,optional,peeled ,Cubed,for garnishing,garnishing,grams,washed, finely chopped,chopped,frying,tablespoon,deseeded,thinly,\
    ,smearing,as per taste,as required, vegans can eliminate this ingredient, roasted,coarsely pounded,\
    ,required,plain,diced,minced,mixed with warm milk,boiled,steamed,as per taste,grated,made into paste,,cooked,\
    taste,(remove seeds fibre any),(Whole),peel cut into rings,(fine variety),cleaned,but recommended,(leftover water from washing rice)\
    to cook,quartered"
    
    stopwords = {'and','as','if','it','at','is','or',"gm","for","ml","whole"}
    
    for i in delword.split(","):
        text = text.replace(i, '')
    text = re.sub(r'\s+', ' ', text)
    text= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text=' '.join(filter(lambda x: x.lower() not in stopwords,  text.split()))
    return text

In [43]:
df1=df.copy()

In [44]:
df1["P-Ingredients"]=df1["TranslatedIngredients"].apply(preprocess)

In [45]:
df1["P-Ingredients"].values

array(['Karela (Bitter Gourd Pavakkai) ,Salt , Onion , Gram flour (besan), Turmeric powder (Haldi), Red Chilli powder, Cumin seeds (Jeera), Coriander Powder (Dhania), Amchur (Dry Mango Powder),Sunflower Oil',
       'rice , tomatoes, BC Belle Bhat powder, salt , chickpea lentils, cumin seeds, white urad dal, mustard, green chilli, dry red chilli, cashew peanuts, oilasafoetida',
       'Rice Vermicelli Noodles (Thin), Onion , Carrots (Gajjar) , Green peas (Matar), Green Chillies, Asafoetida (hing), Mustard seeds, White Urad Dal (Split), Ghee, sprig Curry leaves,Salt , Lemon juice',
       ...,
       'Quinoa, Sugar, Cardamom Powder (Elaichi), Milk vegans can substitute this ingredient with almond milk, Condensed Milk khoya (mawa), Ghee, Saffron strands,Mixed nuts',
       'Spring Onion (Bulb & Greens) , Tamarind Water, Gram flour (besan), Sunflower Oil, Mustard seeds, Cumin seeds (Jeera), Dry Red Chillies, pinch Asafoetida (hing), pinch Turmeric powder (Haldi),Salt to atste, Coriander (

In [46]:
import random
def rd():
    i=random.randint(0,6321)
    x=df1.loc[i,"P-Ingredients"]
    print(x)
    print(i)
    return x

In [47]:
tester=rd()

Chicken chicken gm, Baby corn baby corn gm, Onion , Green Bell Pepper (Capsicum) , Carrot (Gajjar) , Egg,Salt , Red Chilli flakes, Corn flour, White pepper powder, Schezwan sauce, Tomato Ketchup,Sunflower Oil to fry the chicken, Sugar, Button mushrooms , Asparagus , Carrot (Gajjar) , Onion ,Ginger ,Garlic , Fresh Red chillies, Sunflower Oil, Teriyaki Sauce,White pepper powder ,Salt , Fried Rice chicken ,French Fries
5570


In [48]:
df1["flag"]=0

In [49]:
user_ingredients='Gram flour, Curd , Green Chillies made into paste, Lemon juice, Enos fruit salt,Salt , Sunflower Oil, Mustard seeds, Cumin seeds (Jeera), Asafoetida (hing), Green Chillies, Red Chilli powder, Turmeric powder, Jaggery,Salt  , Sunflower Oil, Mustard seeds, Cumin seeds, Asafoetida , Green Chillies, Red Chilli powder, Turmeric, Jaggery , Tamarind ,Salt , Coriander '
len(user_ingredients.split(","))

26

In [50]:
df1.reset_index(drop=True,inplace=True)

In [51]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Srno                    6276 non-null   int64 
 1   TranslatedRecipeName    6276 non-null   object
 2   TranslatedIngredients   6276 non-null   object
 3   PrepTimeInMins          6276 non-null   int64 
 4   CookTimeInMins          6276 non-null   int64 
 5   TotalTimeInMins         6276 non-null   int64 
 6   Servings                6276 non-null   int64 
 7   Cuisine                 6276 non-null   object
 8   Course                  6276 non-null   object
 9   Diet                    6276 non-null   object
 10  TranslatedInstructions  6276 non-null   object
 11  URL                     6276 non-null   object
 12  lang                    6276 non-null   object
 13  P-Ingredients           6276 non-null   object
 14  flag                    6276 non-null   int64 
dtypes: i

In [19]:
rows=6269

In [52]:
def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        return count

61.400166034698486


In [ ]:
df["flag"]=df["P-Ingredients"].apply(Cook2)

In [53]:
df1.loc[93,"P-Ingredients"]

'coconut milk, water, chana lentils, green chillies do more less you wish, oil, mustard, small Spoon turmeric powder, cumin seeds, curry leaves little, salt , coriander leaves little'

In [54]:
df1["flag"].value_counts()

4     654
0     532
6     449
12    429
11    424
8     395
9     394
13    379
10    373
5     352
14    306
7     284
15    260
3     256
16    175
2     171
1     161
17    112
18     84
19     43
20     19
21     13
22      8
24      1
26      1
23      1
Name: flag, dtype: int64

In [55]:
df1.sort_values(by="flag",ascending=False).head(10)

,Srno,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL,lang,P-Ingredients,flag
3293,4502,Rasawala Dhokla Recipe,"2 cups Gram flour (besan),1/2 cup Curd (Dahi /...",5,10,15,4,Gujarati Recipes﻿,North Indian Breakfast,Vegetarian,"To begin making the Rasawala Dhokla recipe, in...",http://www.archanaskitchen.com/rasawala-dhokla...,en,"Gram flour (besan), Curd (Dahi Yogurt), Green ...",26
4847,7665,Maharashtrian Golyachi Amti Recipe (Besan Ball...,"1 cup Gram flour (besan),1 tablespoon Sunflowe...",10,20,30,4,Maharashtrian Recipes,Side Dish,Vegetarian,To Make Balls/Golyachi (kofta)To begin Maharas...,http://www.archanaskitchen.com/maharashtrian-g...,en,"Gram flour (besan), Sunflower Oil, Ajwain (Car...",24
647,714,Dal Dhokli Recipe With Sprouted Moong Dal,"3/4 cup Whole Wheat Flour,1/4 cup Gram flour (...",10,45,55,4,Gujarati Recipes﻿,Lunch,Vegetarian,To begin making the Dal Dhokli Recipe With Spr...,https://www.archanaskitchen.com/dal-dhokli-rec...,en,"Wheat Flour, Gram flour (besan), Red Chilli po...",23
5386,9153,Tindora Ka Salan Recipe,250 grams Tindora (Dondakaya/ Kovakkai) - slit...,10,20,30,4,North Indian Recipes,Side Dish,Vegetarian,"To begin making the Tindora Ka Salan Recipe, d...",https://www.archanaskitchen.com/tindora-ka-sal...,en,Tindora (Dondakaya Kovakkai) slitted keeping t...,22
4304,6410,Andhra Style Mukkala Pulusu Recipe-Mixed Veget...,1/4 cup Mangalorean cucumber - cut into small ...,10,20,30,4,Andhra,Lunch,Vegetarian,"To begin making the Mukkala Pulusu Recipe, fir...",https://www.archanaskitchen.com/mukkala-pulusu...,en,"Mangalorean cucumber cut into small chunks, Ca...",22
302,323,Kurkuri Bhindi With Spicy Tomato Yogurt Gravy ...,"250 grams Bhindi (Lady Finger/Okra),Sunflower ...",20,40,60,4,North Indian Recipes,Lunch,Vegetarian,To begin making the Kurkuri Bhindi With Spicy ...,http://www.archanaskitchen.com/kurkuri-bhindi-...,en,"Bhindi (Lady FingerOkra),Sunflower Oil deep , ...",22
3815,5474,Aloo Bonda Recipe (Deep Fried Snack With Potat...,"5 Potatoes (Aloo) - boiled,2 teaspoon Mustard ...",10,45,55,2,Bengali Recipes,Snack,Vegetarian,"To begin making the Aloo Bonda recipe, firstly...",http://www.archanaskitchen.com/aloo-bonda-reci...,en,"Potatoes (Aloo) , Mustard seeds, Cumin seeds (...",22
6062,12021,Mangalorean Style Bendekayi Huli Koddel Recipe...,"2 cups Bhindi (Lady Finger/Okra),1 cup Yellow ...",10,30,40,4,Mangalorean,Lunch,Vegetarian,To begin making the Mangalorean Style Bendekay...,https://www.archanaskitchen.com/mangalorean-st...,en,"Bhindi (Lady FingerOkra), Yellow Moong Dal (Sp...",22
333,358,Potato Stuffed Dal Dhokli Recipe,"1 cup Whole Wheat Flour,Salt - to taste,1 tabl...",25,75,100,4,North Indian Recipes,Lunch,Vegetarian,"To prepare Potato Stuffed Dal Dhokli Recipe, g...",https://www.archanaskitchen.com/potato-stuffed...,en,"Wheat Flour,Salt , Sunflower Oil,Whole Wheat F...",22
3173,4285,Makai Dhokla Recipe - Savoury Steamed Corn Cake,"1 cup Sweet corn,1 cup Corn meal,1/2 cup Curd ...",20,45,65,4,Gujarati Recipes﻿,Snack,High Protein Vegetarian,"To begin making the Makai Dhokla, firstly crus...",http://www.archanaskitchen.com/makai-dhokla-sa...,en,"Sweet corn, Corn meal, Curd (Dahi Yogurt) whis...",22


In [ ]:
df1.reset_index(drop=True,inplace=True)

In [ ]:
df1.drop([5743,5173,4846,4125,3766,422,80,442,583,4994,5466,5463,5440,5383,5374,4994,1571,3311,3222,2305,1571,605,423],inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [56]:
df1.to_csv('df-en-processed.csv',index=False)

To Preprocess Instructions

In [2]:
df=pd.read_csv("df-en-processed.csv")

In [16]:
df.loc[5,"TranslatedInstructions"]

'To begin making\xa0Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and\xa0Serve the Pudina Khara Pongal Recipe along with\xa0Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and\xa0South Indian Coconut Chutney\xa0for a relaxed weekend breakfast.'

In [23]:
def pre(text):

    delword="\xa0,\n"    
    for i in delword.split(","):
        text = text.replace(i, ' ')
    return text

In [22]:
testtext='To begin making\xa0Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and\xa0Serve the Pudina Khara Pongal Recipe along with\xa0Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and\xa0South Indian Coconut Chutney\xa0for a relaxed weekend breakfast.'

In [24]:
d=pre(testtext)

In [25]:
d

'To begin making Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and Serve the Pudina Khara Pongal Recipe along with Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and South Indian Coconut Chutney for a relaxed weekend breakfast.'

In [26]:
df.columns

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'PrepTimeInMins',
       'CookTimeInMins', 'TotalTimeInMins', 'Servings', 'Cuisine', 'Course',
       'Diet', 'TranslatedInstructions', 'URL', 'lang', 'P-Ingredients',
       'flag'],
      dtype='object')

In [27]:
df["instruc-lang"]=df["TranslatedInstructions"].apply(lambda x:detect(x))

In [28]:
df["instruc-lang"].value_counts()

en    5990
hi     265
Name: instruc-lang, dtype: int64

In [29]:
df=df[df["instruc-lang"]=="en"]

In [31]:
df["TranslatedInstructions"]=df["TranslatedInstructions"].apply(pre)

C:\Users\soory\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df.loc[10,"TranslatedInstructions"]

'To begin making the homemade baked beans recipe, we first have to soak the beans overnight for about 8 hours in water.We also have to make the cooked tomato puree. Click here to see the recipe of how to make tomato puree at home.Once the beans are soaked well, our next step is to cook them. I like using the pressure cooker method to cook the beans. Place the beans in the pressure, place the weight on, cover the pan and cook it along with 2-1/2 cups of water. After you heat 4 to 5 whistles, turn the heat to low and cook for another 15 minutes.Turn off the heat and allow the pressure cooker to release its pressure naturally. This would take another 10 minutes. The beans continue to cook under the pressure even when it is turned off.Once the pressure is released, open the cooker and drain out the excess liquids. You store this liquid and use it in the vegetable stock or risotto.Once the beans are cooked, our next step is to spice it up to make the baked beans.Heat oil in a sauce pan on m

In [35]:
df.to_csv('df-en-processed.csv',index=False)

In [ ]:

df=pd.read_csv("df-en-processed.csv")
user_ingredients="Eggs hard & halved, Onion , Tomato , Coriander Powder (Dhania), Garam masala powder, Red Chilli powder, \
Cinnamon Stick (Dalchini), Cloves (Laung), Sunflower Oil,Salt , Coriander (Dhania) Seeds"


    
    return Cookdf

Cookdf=Cookman(df,user_ingredients)

In [ ]:
def Cookman(df,user_ingredients):
    def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        return count
    df["flag"]=df["P-Ingredients"].apply(Cook2)
    Cookdf=df.sort_values(by="flag",ascending=False).head(10)
    def Missing(Cookdf):
        flag=0
        miss=""
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                miss+=i+","
            miss = miss[:-1]+"."
            else:
                flag=0
        return miss
    Cookdf["Missing"]=Cookdf["P-Ingredients"].apply(Missing)
    return Cookdf

In [46]:
miss=""
i=["hi","bi"]
for b in i:    
   miss+=b+","
miss = miss[:-1]+"."

In [32]:
df=pd.read_csv("df-en-final.csv")

In [24]:
def preprocess(text):
    text=text.lower()
    text=text.strip()
    
    stopwords = {"tsp","tbsp","()"}
    
    text=' '.join(filter(lambda x: x.lower() not in stopwords,  text.split()))

        
    return text

In [25]:
df["P-Ingredients"]=df["P-Ingredients"].apply(preprocess)

In [35]:
df["P-Ingredients"].values

array(['karela (bitter gourd pavakkai) ,salt , onion , gram flour (besan), turmeric powder (haldi), red chilli powder, cumin seeds (jeera), coriander powder (dhania), amchur (dry mango powder),sunflower oil',
       'rice , tomatoes, bc belle bhat powder, salt , chickpea lentils, cumin seeds, white urad dal, mustard, green chilli, dry red chilli, cashew peanuts, oilasafoetida',
       'rice vermicelli noodles (thin), onion , carrots (gajjar) , green peas (matar), green chillies, asafoetida (hing), mustard seeds, white urad dal (split), ghee, sprig curry leaves,salt , lemon juice',
       ...,
       'quinoa, sugar, cardamom powder (elaichi), milk vegans can substitute this ingredient with almond milk, condensed milk khoya (mawa), ghee, saffron strands,mixed nuts',
       'spring onion (bulb & greens) , tamarind water, gram flour (besan), sunflower oil, mustard seeds, cumin seeds (jeera), dry red chillies, pinch asafoetida (hing), pinch turmeric powder (haldi),salt to atste, coriander (

In [27]:
df.to_csv("df-en-final.csv",index=False)

In [49]:
df=pd.read_csv("df-en-final.csv")

In [50]:
def preprocess(text):
    bar=""
    for i in text.split(","):
             bar+=i.strip()+","
    bar=bar[:-1]

        
    return bar

In [51]:
df["P-Ingredients"]=df["P-Ingredients"].apply(preprocess)

In [52]:
df["P-Ingredients"]

0       karela (bitter gourd pavakkai),salt,onion,gram...
1       rice,tomatoes,bc belle bhat powder,salt,chickp...
2       rice vermicelli noodles (thin),onion,carrots (...
3       chicken,onion,tomato,green chillies slit,inch ...
4       chana dal,white urad dal,red chillies,coriande...
                              ...                        
5985    paneer (homemade cottage cheese) crumbled,suga...
5986    risotto risotto (recipe below),parmesan cheese...
5987    quinoa,sugar,cardamom powder (elaichi),milk ve...
5988    spring onion (bulb & greens),tamarind water,gr...
5989    kg chicken medium pieces,mustard oil,ghee,kg c...
Name: P-Ingredients, Length: 5990, dtype: object

In [53]:
df.to_csv("df-en-final.csv",index=False)

In [ ]:
do more less you wish,cut into squares,cut into pieces,cut into cubes,cut into small cubes,per use, coarsely crushed, slit lengthwise,wash well in water, cut into half, make puree,to smear, cut to small florets, soaked in hot water,roughly thin,roughly thin,into inch square pieces,granulated

In [63]:
df=pd.read_csv("df-en-final.csv")

In [59]:
def preprocess(text):
    delword="do more less you wish,tightly packed,cut into squares,cut into pieces,cut into cubes,cut into small cubes,per use, coarsely crushed,slit lengthwise,wash well in water, cut into half, make puree,to smear, cut to small florets, soaked in hot water,roughly thin,roughly thin,into inch square pieces,granulated"
    for i in delword.split(","):
        text = text.replace(i, '')
    text=text.replace("chili","chilli")
    return text

In [60]:
df["P-Ingredients"]=df["P-Ingredients"].apply(preprocess)

In [61]:
df.to_csv("df-en-final.csv",index=False)

In [64]:
df["ingredient-count"]=df["P-Ingredients"].apply(lambda x: len(x.split(",")))

In [68]:
df["ingredient-count"].value_counts()

10    530
12    527
11    511
9     499
13    490
8     429
14    397
15    358
7     342
16    286
6     280
17    232
18    167
5     163
19    150
4     104
20     95
22     77
21     76
23     58
25     36
3      34
24     31
27     28
26     22
29     14
28     12
30      8
31      6
32      5
2       5
33      5
36      3
37      2
34      2
41      1
45      1
39      1
38      1
35      1
43      1
Name: ingredient-count, dtype: int64

In [69]:
df=df[df["ingredient-count"]>3]

In [72]:
df["ingredient-count"].value_counts()

10    530
12    527
11    511
9     499
13    490
8     429
14    397
15    358
7     342
16    286
6     280
17    232
18    167
5     163
19    150
4     104
20     95
22     77
21     76
23     58
25     36
24     31
27     28
26     22
29     14
28     12
30      8
31      6
33      5
32      5
36      3
37      2
34      2
43      1
41      1
39      1
38      1
35      1
45      1
Name: ingredient-count, dtype: int64

In [74]:
df.reset_index(drop=True,inplace=True)

In [36]:
df.to_csv("df-en-final.csv",index=False)

In [37]:
df=pd.read_csv("df-en-final.csv")

In [38]:
df["P-Ingredients"]

0       karela (bitter gourd pavakkai),salt,onion,gram...
1       rice,tomatoes,bc belle bhat powder,salt,chickp...
2       rice vermicelli noodles (thin),onion,carrots (...
3       chicken,onion,tomato,green chillies slit,ginge...
4       chana dal,white urad dal,red chillies,coriande...
                              ...                        
5946    paneer  crumbled,sugar,cardamom powder (elaich...
5947    risotto risotto (recipe below),parmesan cheese...
5948    quinoa,sugar,cardamom powder (elaichi),milk ve...
5949    spring onion (bulb  greens),tamarind water,gra...
5950    chicken medium pieces,mustard oil,ghee,curd (d...
Name: P-Ingredients, Length: 5951, dtype: object

In [39]:
def prep(text):
    delword="cut into julienne,in strips,(),( ), cut in two pieces,to grease the cake tin,cut in cubes,(any cheese of your choice),small size but not baby potatoes,kg \
,cut into long strips,(or any flavour you prefer),few,cut into strips thenfried in oil,( + ),only filled shaped,to make pessaratu,to roast the parathas,thin long,any alcohol of choice \
 ,broken into halves,+ egg egg wash,thin diagonally,cut into inch sticks,cut into small pieces,to garnish,into small pieces,more  while cooking \
 ,hot water minutes soak in,&,cut into wedges,(or flax meal egg replacer),podsseeds with seeds removed,(lengthwise preferably) \
 ,to serve,cut into slices,extra,all,small bunch,deep,(homemade cottage cheese),pinch,powdered,garnish, to cook,cooking, per your taste \
 ,hot,roughly torn,inch,fresh,pinch,ssp,soaked hours,according,with stems removed,more,cubed,per your taste,more"
    for i in delword.split(","):
        text = text.replace(i, '')   
    text=text.replace("onions","onion")
    text=text.replace("salt salt","salt")
    return text

In [40]:
df["P-Ingredients"]=df["P-Ingredients"].apply(prep)

In [41]:
df["P-Ingredients"]

0       karela (bitter gourd pavakkai),salt,onion,gram...
1       rice,tomatoes,bc belle bhat powder,salt,chickp...
2       rice vermicelli noodles (thin),onion,carrots (...
3       chicken,onion,tomato,green chillies slit,ginge...
4       chana dal,white urad dal,red chillies,coriande...
                              ...                        
5946    paneer  crumbled,sugar,cardamom powder (elaich...
5947    risotto risotto (recipe below),parmesan cheese...
5948    quinoa,sugar,cardamom powder (elaichi),milk ve...
5949    spring onion (bulb  greens),tamarind water,gra...
5950    chicken medium pieces,mustard oil,ghee,curd (d...
Name: P-Ingredients, Length: 5951, dtype: object

In [42]:
def preprocess(text):
    bar=""
    for i in text.split(","):
             bar+=i.strip()+","
    bar=bar[:-1]

        
    return bar

In [43]:
df["P-Ingredients"]=df["P-Ingredients"].apply(preprocess)

In [44]:
df["P-Ingredients"].sample(10).values

array(['sesame seeds (til seeds),raw peanuts (moongphali),baby potatoes,salt,sunflower oil,cumin seeds (jeera),green chillies slit,ginger,cloves garlic,cumin powder (jeera),red chilli powder,garam masala powder,lemon juice,coriander (dhania) leaves,mint leaves (pudina),chaat masala powder',
       'masoor dal soaked,onion,tomato,green chillies slit,asafoetida (hing),bay leaf (tej patta),cumin seeds (jeera),clove garlic,ginger inely,red chilli powder,coriander powder (dhania),turmeric powder (haldi),garam masala powder,sprig coriander (dhania) leaves,salt,sunflower oil',
       'bhindi (lady fingerokra) cut into  pieces,onion big,green chilli slit,coconut milk thick,ginger,mustard seeds,sprig curry leaves,coconut oil,salt',
       'basmati rice,yellow moong dal (split),potatoes (aloo),onion,green peas (matar),coconut,sugar,cloves (laung),cinnamon stick (dalchini),bay leaf (tej patta),cardamom (elaichi) podsseeds,cumin seeds (jeera),turmeric powder (haldi),green chillies,cloves garlic,gi

In [45]:
df.to_csv("df-en-final.csv",index=False)

In [30]:
df=pd.read_csv("df-en-final.csv")

In [47]:
im=pd.read_csv("images.csv")

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5951 entries, 0 to 5950
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   TranslatedRecipeName    5951 non-null   object
 1   TranslatedIngredients   5951 non-null   object
 2   TotalTimeInMins         5951 non-null   int64 
 3   Cuisine                 5951 non-null   object
 4   TranslatedInstructions  5951 non-null   object
 5   URL                     5951 non-null   object
 6   P-Ingredients           5951 non-null   object
 7   flag                    5951 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 372.1+ KB


In [53]:
df["URL"]

0       https://www.archanaskitchen.com/masala-karela-...
1       http://www.archanaskitchen.com/spicy-tomato-ri...
2       http://www.archanaskitchen.com/ragi-vermicelli...
3       http://www.archanaskitchen.com/gongura-chicken...
4       https://www.archanaskitchen.com/andhra-style-a...
                              ...                        
5946    https://www.archanaskitchen.com/saffron-paneer...
5947    http://www.archanaskitchen.com/italian-arancin...
5948    https://www.archanaskitchen.com/quinoa-phirnee...
5949    https://www.archanaskitchen.com/ullikadala-pul...
5950    http://www.archanaskitchen.com/kashmiri-kokur-...
Name: URL, Length: 5951, dtype: object

In [54]:
im.head()

,image,url
0,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/masala-karela-...
1,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/spicy-tomato-r...
2,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/ragi-vermicell...
3,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/gongura-chicke...
4,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/andhra-style-a...


In [55]:
df["image-url"]=im["image"]

In [57]:
df["image-url"].values

array(['https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Pooja_Thakur/Karela_Masala_Recipe-4_1600.jpg',
       'https://www.archanaskitchen.com/images/archanaskitchen/1-Author/b.yojana-gmail.com/Spicy_Thakkali_Rice_Tomato_Pulihora-1_edited.jpg',
       'https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Ragi_vermicilli.jpg',
       ...,
       'https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Reshma_Seetharam/Quinoa_kheer_Phirni.jpg',
       'https://www.archanaskitchen.com/images/archanaskitchen/1-Author/anu.mallajosyula-gmail.com/Ullikadala_Pulusu_Spring_Onion_pulusu.jpg',
       'https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Shaheen_Ali/Kokur_Yakhni_Chicken_cooked_in_Yogurt__Kashmiri_Non_Veg.jpg'],
      dtype=object)

In [58]:
df["URL"]=im["url"]

In [62]:
df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,P-Ingredients,flag,image-url
0,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"karela (bitter gourd pavakkai),salt,onion,gram...",-5,https://www.archanaskitchen.com/images/archana...
1,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"rice,tomatoes,bc belle bhat powder,salt,chickp...",-9,https://www.archanaskitchen.com/images/archana...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"rice vermicelli noodles (thin),onion,carrots (...",-10,https://www.archanaskitchen.com/images/archana...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"chicken,onion,tomato,green chillies slit,ginge...",-11,https://www.archanaskitchen.com/images/archana...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"chana dal,white urad dal,red chillies,coriande...",-8,https://www.archanaskitchen.com/images/archana...


In [61]:
df.to_csv("df-en-final.csv")

In [19]:
df["image-url"].nunique()

5666

In [25]:
df[df["image-url"]= https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Nandan_Kozhi__Kerala_Style_Chicken_Curry.jpg

SyntaxError: invalid syntax (<ipython-input-25-97055e7bcdf9>, line 1)

In [32]:
df["image-url"].value_counts()

https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Nandan_Kozhi__Kerala_Style_Chicken_Curry.jpg                                                4
https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Preeti_Tamilarasan/Phalguni_Dal_Recipe_original_1600.jpg                                                     3
https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2016/sept-21/Peanut_Coconut_Chutney_Recipe_No_onion_No_Garlic_Fasting_Vrat_Recipe-3.jpg    3
https://www.archanaskitchen.com/images/archanaskitchen/0-Affiliate-Articles/sweet_ime_soda.jpg                                                                               3
https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2019/Veg_Hakka_Noodles_recipe_10_1600.jpg                                                  3
                                                                                                                             

In [36]:
df[df["image-url"]== "https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Nandan_Kozhi__Kerala_Style_Chicken_Curry.jpg"]

,Unnamed: 0,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,P-Ingredients,flag,image-url
398,398,Kerala Chicken Curry Recipe With Freshly Groun...,"500 Chicken - medium sized pieces,1 Onion - ro...",55,Kerala Recipes,To prepare Kerala Chicken Curry With Freshly G...,https://www.archanaskitchen.com/kerala-chicken...,"chicken medium sized pieces,onion roughly,ging...",-10,https://www.archanaskitchen.com/images/archana...
2228,2228,Kerala Chicken Curry Recipe - Nadan Kozhi Curry,"500 grams Chicken - curry-cut with bones,2 tea...",40,Kerala Recipes,"To begin making Kerala Chicken Curry Recipe, w...",https://www.archanaskitchen.com/kerala-chicken...,"chicken currycut with bones,red chilli powder,...",-9,https://www.archanaskitchen.com/images/archana...
3353,3353,Nandan Kozhi Recipe - Kerala Style Chicken Curry,"500 grams Chicken - curry cut,3 Onions - slice...",50,Kerala Recipes,"To begin making the Nandan Kozhi recipe, first...",https://www.archanaskitchen.com/nandan-kozhi-r...,"chicken curry cut,onion,tomatoes,ginger,cloves...",-8,https://www.archanaskitchen.com/images/archana...
3356,3356,Kerala Chicken Curry Recipe,400 grams Chicken - cut into medium sized piec...,70,Kerala Recipes,"To prepare the Kerala Chicken Curry, in a mixi...",https://www.archanaskitchen.com/kerala-chicken...,"chicken cut into medium sized pieces,curd (dah...",-11,https://www.archanaskitchen.com/images/archana...


In [39]:
def imc(text):
    if "https" in text:
        return 1
    else:
        
        return 0
df["imcheck"]=df["image-url"].apply(imc)

In [693]:
df=pd.read_csv("df-en-final.csv")

In [694]:
df["P-Ingredients"]

0       karela (bitter gourd pavakkai),salt,onion,gram...
1       rice,tomato,bc belle bhat powder,salt,chickpea...
2       rice vermicelli noodles (thin),onion,carrots (...
3       chicken,onion,tomato,green chillies,ginger,clo...
4       chana dal,white urad dal,red chillies,coriande...
                              ...                        
5946    paneer,sugar,cardamom powder (elaichi),saffron...
5947    risotto,parmesan cheese,gorgonzola cheese cube...
5948    quinoa,sugar,cardamom powder (elaichi),milk,co...
5949    spring onion (bulb greens),tamarind water,gram...
5950    chicken,mustard oil,ghee,curd,cinnamon stick (...
Name: P-Ingredients, Length: 5951, dtype: object

In [695]:
def preprocess(text):

    delword="julienned,overnight,minutes,dressing,in cubes,in water least hours,(or any other herb of choice),peeled,cut into cubes,beaten with little water \
    ,kneading,(low fat),cut into sm piece,(for ),to make sandwiches,add you like loose gravy,leaves sm bunch,lukewarm,cut into florets \
    ,seasoning,handful,halved,broken,beaten,softened,room temperature,cut into segments,cut into diagonals,squeezed into juice \
    ,soaked,clean,roughly,neat,mashed,crushed,toasted,lightly,coarsely,( hrs.),sm bunch,taste,pounded,spread,long,slit,tighten,spread,little,torn \
    ,adjustable,sized,(you can also use irish cream milk),adjust spice threshold,(unsalted),active,( divided into two equal portions) \
    ,( divided into two equal portions),(canned ),(adjust),(dahi yogurt),make the batter,cut into strips,cut into sm florets,warm,lukewarm \
    ,one head,(% fat will do),very thin,dessicated,leaves bunch,(or ),adjust,chilled,dusting,(used ),(depending upon how spicy you want) \
    ,(preferably with skin),( tbsp),melted,hours,made,your choice,greasing pan,crumbled,cut into angular slices,strands in milk,(use % fat),liter \
    ,(veganalmond milk instead),purpose,into puree,tablespoon,cut into triangle,in water,(used large ones),cut diagony,pureecut into sticks \
    ,(omit ginger jain),( greasing palms plate),( based on pungency),shelled deveined,soaked,clean,neat,roughly,rough,mashed,crushed,toasted,lighltly \
    ,(hrs.),coarsely,roughly,sm bunch,tighten,slit,pounded,little,long,(or large onion),(about gram)\
    , reduce bitter from bitter gourd,not available,wash,cut one cubes,removed,cut  thin wedges with seeds removed \
    ,kaffir lime leaves you can,home,into halves,to pieces,blanched,(rind),fine variety,roasted,pureed,you can thin,(optional) \
    ,(used % cocoa),(dried raw green mangoes easily available indian stores),like juliennes,pitted,making dosa,big,(or rock candy sugar)\
    ,much,grind smooth paste,clarified butter,(can reduce ½ your mangoes are very sweet),( ),slivered,into chunks \
    ,into cubes,make powder,greasing the breads,( you like sweetness),(kala namak),into diamonds,goat meat with bone \
    ,(like adding less sugar please your ),ghee coat the pan ( you can an oil spray),gooseberry size,one bunch,whisked,into medium cubes \
    (or combination flours),chunks,drained,rinsed,into florets,(add serving kids crowd with sweet tooth),into rings,into round slices \
    into two halves,serving,bite size,sprinkle,until soft,hard,yolks,thick,whites,boil peel,less too sour,boil boil,(you can substitute with palm sugar) \
    ,with leaf half will also do,with leaf half will also do,clotted,add,chopped,four pieces,grease the mould,half the size lemon \
    ,toast the burger buns,(divided),while,vegans can substitute this ingredient with almond milk,brioche bread preferred,cup,store bought. \
    mildly soured,(clotted', '),per the experiment,also ced sago,(reserve ),(as ),(soak ),other water fish,peel,½ pieces,ghee dosa,reserve \
    ,* pices, * rectangles,cm pieces,your,plus ghee (vegans can coconut oil),(this makes the meat tender),ends,' pieces,trained remove lumps \
    ,fried,tightly,(sour),( large) scrubbed,stock ed,yogurt –,(optional just give ness gravy),(sm),बड़े चमच्च हींग,per need,full fat \
    ,you can skip this rice,(or make glaze),(to rub hands before shaping peda),quarters,per you,(or make firm dough),large dice,length wise \
    ,½ buttermilk,coating,drizzle,low fat,(refer notes),( reduce quantity ),bite pieces,bite pieces,compress take coriander leaves \
    ,picked,refrigerated,atste,half half (,(reduce quantity ),tate,make sure the stalks have been only leaves used,out the seeds,coursely \
    , other fruits are also,sour variety,(prefer ),the dosa,the sandwiches,tenderize the chicken,straight,liced,hin roun dels,from the middle \
    ,till golden brown,pieces,about,medium cubes,( spice preference),(cut ),petals,(or same quantity onion tomato),(sambar onion) (sambar onion) \
    ,half,rings,until brown,pasted,one,large pieces,(or half ghee half oil),one squares,circles grilled,grilled,(sambar onion) regular onion \
    ,wedges,partly,thingly,½,(lengthwise preferably),rounds,half layers separated,(sambar onion)or regular onion',(about ),caramelized \
    ,medium,refrigerated,keep the dough moist,crispy,make paste,roundels,(if using unsalted juice),( the heat the chilli),directly white \
    ,pickled,tossing,half,sauteed ground,thin stripes, straight thin,(sambar onion) red onion,thin rings,caramelized,bottom take"
    
    stopwords = {"'",".","each","steam","fine","thin","sk","es","cas","very","round","liter","size","+","'s","whisk","(with full,)","s","fry","beat","( )","=","into cubes","cup","cups","with","into","()","add","slices","raw","to","lengthwise","in","show","soaked","clean","neat","rough","mashed","crushed","toasted","lightly","( hrs.)","coarsely","roughly",
                 "of","to use","slit","cted","bits","sm bunch","taste","puree","tighten","torn","spread","pounded","little","long","ss","ct"
                 ,"b","sm","cpu","ll","cd","per your","use","in""tsp","any","d","ly","+","chop","well","cut",'and','as','if','it','at','is','or',"gm","for","ml","whole","puree"}    
    


    
    text=text.replace("lemon juice from lemons","lemon juice")
    text=text.replace("ripe bananas","bananas")
    text=text.replace("water water","water")
    text=text.replace("'dry yeast dry yeast","dry yeast")
    text=text.replace("milk)","milk")
    text=text.replace("sunflower oil the onion","sunflower oil, onion")
    text=text.replace("oniononion","onion")
    text=text.replace("oniononion","onion")
    text=text.replace("onion take green chillies","onion,green chillies")
    text=text.replace("sprigs mintonionand cook till turns brown.","sprigs mint,onion")
    text=text.replace("mixed vegetables onion","mixed vegetables,onion")
    text=text.replace("onion ground paste","onion")
    text=text.replace("(pearl onion)","onion")
    text=text.replace("water onion","water,onion")
    text=text.replace("milk)","milk")
    text=text.replace("large onion","onion")
    text=text.replace("onionand","onion")
    text=text.replace("lemon juice curd","lemon juice")
    text=text.replace("salt kala namak","salt")
    text=text.replace("orange zest","oranges")
    text=text.replace("milk milk","milk")
    text=text.replace("lemon zest","lemon")
    text=text.replace("salt pepper","salt,pepper")
    text=text.replace("sugar sugar","sugar")
    text=text.replace("sugar honey","honey")
    text=text.replace("tomatoesor","tomato")
    text=text.replace("tomatoes","tomato")
    text=text.replace("onionfor","onion")
    text=text.replace("coconut coconut","cocunut")
    text=text.replace("garam masala powderof garam masala","garam masala")
    text=text.replace("egg eggs depending on how many tarts you have prepared","eggs")
    text=text.replace("salt you are using unsalted butter","butter")
    text=text.replace("lemon juice can add lemon","lemon")
    text=text.replace("sugar ground","sugar")
    text=text.replace("milk preferably ‘whole milk’","milk")
    text=text.replace("water leftover water chana","water")
    text=text.replace("cooked rice","rice")
    text=text.replace("oranges","orange")
    text=text.replace("brush before baking","")
    text=text.replace("tomato grated","tomato")
    text=text.replace("lemons juiced","lemon juice")
    text=text.replace("lemon juice squeezed","lemon juice")
    text=text.replace("risotto risotto (recipe below)","risotto")
    text=text.replace("broccoli head","broccoli")
    text=text.replace("badam (almond) slices","badam (almond)")
    text=text.replace("sunflower oil fry","sunflower oil")
    text=text.replace("rock salt","salt")
    text=text.replace("sea salt","salt")
   
    text=text.split(",")
    x=""
    for i in text:
          resultwords = [word for word in i.split() if word not in stopwords]
          result = " ".join(resultwords)
          x+=","+result
    text=x[1:]
    for i in delword.split(","):
         text = text.replace(i, '')
    bar=""   
    for i in text.split(","):
             bar+=i.strip()+","
    bar=bar[:-1]
   
    bar=bar.replace("cream",",")
    bar=bar.replace(",,",",")
    bar=bar.replace(", ,",",")
    return bar

In [696]:
df["P-Ingredients"]=df["P-Ingredients"].apply(preprocess)

In [697]:
df["P-Ingredients"].sample(10).values

array(['wheat flour,ragi flour (finger millet nagli),instant oats (oatmeal),sesame seeds (til seeds),water ( degrees),hy,dry yeast,salt,virgin olive oil,pizza pasta sauce,mozzarella cheese,button mushrooms,green bell pepper (capsicum),green zucchini,dried oregano pizza,salt,pepper',
       'carrot,potatoes potato,suran,green peas,raisins,dates,horse,onion,tomato,garlic,ginger,green chillies,turmeric powder,red chilli powder,dhansak masala,salt,oil',
       'mixed vegetables frozen,onion,cloves garlic,tomato,pasta wheat rotini,kabuli chana (white chickpeas) un,tomato ketchup,dried basil leaves,cinnamon powder (dalchini),paprika powder red chilli flakes,black peppercorns,turmeric powder (haldi),salt,water,virgin olive oil',
       'wheat flour,dry yeast,water,sugar,curd,salt',
       'sprig tulsi (holy basil),betel leaves (paan),cloves garlic,ajwain (carom seeds),black pepper powder,cumin seeds (jeera),tamarind,tomato,curry leaves,ginger,asafoetida (hing),salt,turmeric powder (haldi),wat

In [698]:
df.to_csv("df-en-final.csv")

In [651]:
count={}
for i in range(len(df)):
    for i in df.loc[i,"P-Ingredients"].split(","):
        if i in count.keys():
            count[str(i)]+=1
        else:
            count[str(i)]=1  

In [652]:
len(count)

6078

In [653]:
dict(sorted(count.items(), key=lambda item: item[1] ,reverse=True))

{'salt': 5410,
 'sunflower oil': 2252,
 'onion': 2222,
 'turmeric powder (haldi)': 1917,
 'red chilli powder': 1693,
 'cloves garlic': 1547,
 'ginger': 1400,
 'tomato': 1329,
 'cumin seeds (jeera)': 1309,
 'green chillies': 1268,
 'water': 1114,
 'mustard seeds': 1077,
 'sugar': 1056,
 'ghee': 856,
 'asafoetida (hing)': 854,
 'coriander (dhania) leaves': 812,
 'garam masala powder': 809,
 'black pepper powder': 800,
 'lemon juice': 788,
 'sprig curry leaves': 787,
 'coconut': 764,
 'coriander powder (dhania)': 732,
 'virgin olive oil': 705,
 'cumin powder (jeera)': 654,
 'dry red chillies': 653,
 'green chilli': 648,
 'butter': 634,
 'milk': 556,
 'curd': 548,
 'cinnamon stick (dalchini)': 508,
 'flour (maida)': 502,
 'wheat flour': 460,
 'black peppercorns': 451,
 'sprig coriander (dhania) leaves': 449,
 'cloves (laung)': 428,
 'cardamom (elaichi) podsseeds': 426,
 'jaggery': 408,
 'curry leaves': 373,
 'white urad dal (split)': 366,
 'fennel seeds (saunf)': 322,
 'gram flour (besan)'

In [317]:
    text="thai red a chilli (birds eye chilli),thai red chilli (birds eye chilli)"
    text=text.split(",")
    x=""
    for i in text:
          resultwords = [word for word in i.split() if word not in stopwords]
          result = " ".join(resultwords)
          x+=","+result
    text=x[1:]
    text

'thai red chilli (birds eye chilli),thai red chilli (birds eye chilli)'